In [8]:
import pandas as pd
import string
import re
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
import numpy as np
import nltk
import csv
from nltk.corpus import stopwords
# nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer 
ps=PorterStemmer()

import tensorflow as tf
import torch
from transformers import BertTokenizer, BertModel
from transformers import AutoModel, AutoTokenizer
from collections import defaultdict
import json

"""
2 corpora - before and after. split().-> bag of words - remove their names to @user for all of them within the content of the tweet not the person who tweeted. Pull the pre-trained bert vectors - hugging face - interate through the list - get the vectors of each words, dont remove duplicates (if dont find vectors for a word - have an output for those words - see if theres any spelling). we can also lemantize them, try to get hte vectors for those lemna, if still no vector - output, keep them somewhere. 
Dictionary: {Day: {tweet_id: {word}: []}, {word []}, etc.}

Once we have the list of words of dehumanizing language, count/number of words in your corpus: (e.g.: 2% were dehumanizing before, 7% after, frequency for each day and have a time-series analysis)
"""

'\n2 corpora - before and after. split().-> bag of words - remove their names to @user for all of them within the content of the tweet not the person who tweeted. Pull the pre-trained bert vectors - hugging face - interate through the list - get the vectors of each words, dont remove duplicates (if dont find vectors for a word - have an output for those words - see if theres any spelling). we can also lemantize them, try to get hte vectors for those lemna, if still no vector - output, keep them somewhere. \nDictionary: {Day: {tweet_id: {word}: []}, {word []}, etc.}\n\nOnce we have the list of words of dehumanizing language, count/number of words in your corpus: (e.g.: 2% were dehumanizing before, 7% after, frequency for each day and have a time-series analysis)\n'

In [2]:
def remove_stop_words(text: string):
    return ' '.join([item for item in text.split() if item not in stopwords.words('english')])

def remove_punctuations(text: string):
    return "".join([char for char in text if char not in string.punctuation])

def stem_text(text: string):
    return [ps.stem(word) for word in text]

def delete_usernames(text: str) -> str:
    pattern = r'@\w+'
    return re.sub(pattern, '@user', text)

In [3]:
before_corpus_list = []
after_corpus_list = []

tweets = {}

with open("Pre-processing.csv", 'r', newline='') as csvFile:
    reader = csv.reader(csvFile)
    next(reader) #skip header
    for line in reader:
        person = line[0].strip()
        tweets[person] = {}

        b_tweet = delete_usernames(remove_punctuations(remove_stop_words(line[3].strip())))
        a_tweet = delete_usernames(remove_punctuations(remove_stop_words(line[4].strip())))

        before_corpus_list.append(b_tweet)
        after_corpus_list.append(a_tweet)
        tweets[person]["Before"] = b_tweet.lower()
        tweets[person]["After"] = a_tweet.lower()


In [7]:
# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_word_embeddings(sentence):
    # Tokenize the sentence
    encoded_input = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True, max_length=512)
    
    # Get the BERT embeddings
    with torch.no_grad():
        output = model(**encoded_input)
    
    # Get word-level embeddings
    token_embeddings = output.last_hidden_state[0].numpy()
    
    # Map tokens back to original words
    word_vectors = {}
    words_without_vectors = set()
    words = sentence.split()
    token_ids = encoded_input['input_ids'][0]
    tokens = tokenizer.convert_ids_to_tokens(token_ids)
    
    word_index = 0
    for i, token in enumerate(tokens):
        if token.startswith('##'):
            if word_index > 0:
                word_vectors[words[word_index-1]] = \
                    (word_vectors[words[word_index-1]] * (i-start) + token_embeddings[i]) / (i-start+1)
        elif token in ['[CLS]', '[SEP]', '[PAD]']:
            continue
        else:
            if word_index < len(words):
                word_vectors[words[word_index]] = token_embeddings[i]
                start = i
                word_index += 1

    words_without_vectors = set(words) - set(word_vectors.keys())
    return word_vectors, words_without_vectors

# Process tweets and generate word vectors
result = defaultdict(dict)
words_without_vectors_global = set()

for person, person_tweets in tweets.items():
    for tweet_type in ["Before", "After"]:
        result[person][tweet_type], words_without_vectors = get_word_embeddings(person_tweets[tweet_type])
        words_without_vectors_global.update(words_without_vectors)

# Print a sample of the result
for person in list(result.keys())[:2]:  # Print for first 2 persons
    print(f"\n{person}:")
    for tweet_type in ["Before", "After"]:
        print(f"  {tweet_type}:")
        for word, vector in list(result[person][tweet_type].items())[:2]:  # Print first 2 words
            print(f"    {word}: {vector[:5]}...")  # Print first 5 values of each vector
        if len(result[person][tweet_type]) > 2:
            print("    ...")

print("\nWords without output vectors:")
print(sorted(list(words_without_vectors_global)))
print(f"Total words without vectors: {len(words_without_vectors_global)}")


RealSpikeCohen:
  Before:
    just: [-0.6662592   0.01738216  0.49830064  0.04646187  0.696845  ]...
    month: [-0.64293134 -0.33376592  0.37095174 -0.09896091  0.30240285]...
    ...
  After:
    hamas: [-0.58247954  0.08837597  0.62763894 -0.31676415  0.17383316]...
    great: [ 0.28736863  0.07461813  0.68630224 -0.4492811  -0.11793958]...
    ...

ComicDaveSmith:
  Before:
    if: [-0.00075689  0.15637913  0.17640428 -0.15965573  0.06332569]...
    mission: [ 0.5582286  -0.18407351  0.16965123  0.10473238  0.34409454]...
    ...
  After:
    as: [-0.5240849   0.34253213  1.6846156   0.37956646  0.01245819]...
    israeli: [-0.27700266  0.26969013  0.09555568 -0.62538004  0.31419805]...
    ...

Words without output vectors:
['2005', '250', '7', 'according', 'actually', 'again', 'asinineisrael', 'assad', 'awful', 'benny', 'brutal', 'calling', 'centerleft', 'chemical', 'chief', 'clearly', 'comparison', 'convincing', 'democraticallyelected', 'determine', 'dictator', 'did', 'differen

In [5]:
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

dumped = json.dumps(result, cls=NumpyEncoder)

with open('Tokens.json', 'w') as fp:
    json.dump(dumped, fp)

In [11]:
#DIFFERENT BERT MODEL FOR TWEETS

bertweet = AutoModel.from_pretrained("vinai/bertweet-base")

# For transformers v4.x+:
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)

# For transformers v3.x:
# tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base")

# INPUT TWEET IS ALREADY NORMALIZED!
line = "SC has first two presumptive cases of coronavirus , DHEC confirms HTTPURL via @USER :cry:"

input_ids = torch.tensor([tokenizer.encode(line)])

with torch.no_grad():
    features = bertweet(input_ids)  # Models outputs are now tuples
    
# Print the output (features)
print("Model output (features):")
print(features)

# If you want to print specific parts of the output, you can do so like this:
print("\nLast hidden state shape:")
print(features.last_hidden_state.shape)
print("\nFirst few values of the last hidden state:")
print(features.last_hidden_state[0, 0, :10])  # First 10 values of the first token

Model output (features):
BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.0261,  0.2147,  0.1159,  ...,  0.0314,  0.0336, -0.1419],
         [ 0.0986, -0.0205,  0.2265,  ...,  0.0828, -0.2993,  0.4767],
         [-0.1011,  0.2133, -0.2283,  ...,  0.0797, -0.1762, -0.2632],
         ...,
         [-0.3448, -0.2996, -0.2430,  ..., -0.3354,  0.2429,  0.3029],
         [-0.5279, -0.2429,  0.0758,  ..., -0.1733,  0.0389,  0.0513],
         [-0.0425,  0.2355,  0.1219,  ...,  0.0018,  0.0732, -0.1305]]]), pooler_output=tensor([[ 2.1684e-01, -1.6747e-01, -3.9913e-02, -1.6048e-01,  1.2537e-01,
         -7.5435e-02,  2.1886e-01, -1.3510e-01,  2.0169e-01, -2.1097e-01,
         -9.4783e-02, -5.3385e-02, -2.2015e-01,  1.3985e-02,  1.5082e-01,
         -6.2459e-02, -1.4716e-01, -6.7477e-02,  2.4149e-02,  1.7152e-01,
         -1.1483e-01, -1.7264e-01,  2.9670e-01, -1.6612e-02, -2.3634e-02,
          3.5212e-02, -1.6087e-01, -3.7130e-02,  4.9394e-02, -5.8116e-02,
         -5